In [1]:
import pandas as pd
from pandas import option_context

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import re
import string

In [2]:
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tawneykirkland/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Main notebook for pre-processing tweets

### Prepare dataframe

In [2]:
df = pd.read_csv('custom_out2.csv')
df.dropna(axis=1,inplace=True)

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df['year'] = pd.DatetimeIndex(df['date']).year

## Filter down to non-Revolut tweets (intention is to understand tweets from customers)
df_eng = df[(df['username'] != 'revolutapp') & (df['language'] == 'en') & (df['year'] == 2020)]

### Clean tweet column

In [10]:
def clean_text(text):
    '''Make text lowercase, remove punctuation, remove reference to @revolutapp,remove links and mentions'''
    text = text.lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = re.sub('[0-9\n]',' ',text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('revolutapp', '', text)
    
    return text

df_eng['tweet'] = df_eng.tweet.map(clean_text)

In [11]:
## Remove emojis
df_eng['tweet'] = df_eng['tweet'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [12]:
df_eng.shape

(110317, 23)

In [7]:
with option_context('display.max_colwidth', 600):
    display(df.sample(10))

,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,language,...,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,video,reply_to,year
164341,1258370884248563712,1258352012917460994,2020-05-07 08:19:32 EDT,2020-05-07,08:19:32,-500,468596733,fulifulibang,i see your point,en,...,0,0,0,[],[],https://twitter.com/fulifulibang/status/1258370884248563712,False,0,"[{'screen_name': 'ploutonios89', 'name': 'X-MAN', 'id': '1220711335689846784'}, {'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]",2020
197963,1236296738580975618,1236296738580975618,2020-03-07 09:24:46 EST,2020-03-07,09:24:46,-500,1205998930019995648,nana38221658,my fucking money unlock my account i need my money,en,...,3,0,2,[],[],https://twitter.com/Nana38221658/status/1236296738580975618,False,0,[],2020
183861,1244923552735920128,1244922762587770880,2020-03-31 05:44:39 EDT,2020-03-31,05:44:39,-500,1244922361054461952,adam_oualim,i did it no answer,en,...,1,0,0,[],[],https://twitter.com/adam_oualim/status/1244923552735920128,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]",2020
116190,1285207304141443072,1285205816002437126,2020-07-20 09:37:53 EDT,2020-07-20,09:37:53,-500,2676313880,revolutapp,great weve already sent you a dm,en,...,0,0,0,[],[],https://twitter.com/RevolutApp/status/1285207304141443072,False,0,"[{'screen_name': 'MateoTillmann', 'name': 'Tillmann Mateo', 'id': '1246358625490214912'}]",2020
97793,1296061603620757504,1296061603620757504,2020-08-19 08:29:00 EDT,2020-08-19,08:29:00,-500,1962704155,altfinews,it was bizarre working remotely and doing a deal from my kitchen table tom hambrett told altfi,en,...,0,0,1,[],[],https://twitter.com/AltFiNews/status/1296061603620757504,False,0,[],2020
103157,1292890747428110340,1292883855637086209,2020-08-10 14:29:09 EDT,2020-08-10,14:29:09,-500,2757074278,brothervader,why didnt you use autobuy yes i mean bitcoin,en,...,0,0,0,[],[],https://twitter.com/brothervader/status/1292890747428110340,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]",2020
36634,1338020184355819520,1337965470490619906,2020-12-13 02:17:25 EST,2020-12-13,02:17:25,-500,1316213479783559168,superwar224,ardcoin ardx is an innovative blockchain based loyalty program that has been integrated into a mobile super wallet which is backed by an existingscn,en,...,0,0,0,['scn1'],[],https://twitter.com/SuperWar224/status/1338020184355819520,False,0,"[{'screen_name': 'SmartCryptoNew1', 'name': '⚡️Smart Crypto News ⚡️', 'id': '1252722861002813440'}, {'screen_name': 'ArkEcosystem', 'name': 'ARK.io', 'id': '774791455680434176'}, {'screen_name': 'rabona_io', 'name': 'Rabona', 'id': '1286158334794649601'}, {'screen_name': 'ard_coin', 'name': 'ArdCoin', 'id': '1021578677614563328'}, {'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}, {'screen_name': 'ExchaseF', 'name': 'Exchase FinTech', 'id': '1323312473215242240'}]",2020
214388,1225412912686227457,1225412912686227457,2020-02-06 08:36:20 EST,2020-02-06,08:36:20,-500,1224798279742959619,aida00485910,hi ive been trying to solve a problem with my app for a few days but for some reason your help center stopped replying to my messages i cant even check my balance please get back to me asap thank you,en,...,1,0,0,[],[],https://twitter.com/Aida00485910/status/1225412912686227457,False,0,[],2020
157986,1262744168616452097,1262743731267989506,2020-05-19 09:57:25 EDT,2020-05-19,09:57:25,-500,2676313880,revolutapp,hi denise weve just left you a message please check your inbox and lets look into this,en,...,0,0,0,[],[],https://twitter.com/RevolutApp/status/1262744168616452097,False,0,"[{'screen_name': 'ThisIrishFamily', 'name': 'Denise 🇺🇸 🍀', 'id': '182579555'}]",2020
197498,1236902438881447936,1236830905303470080,2020-03-09 02:31:36 EDT,2020-03-09,02:31:36,-500,2676313880,revolutapp,hey there our message has been sent to you aurelia,en,...,0,0,0,[],[],https://twitter.com/RevolutApp/status/1

In [27]:
## Remove one and two word tweets
one_word = df_eng[df_eng['tweet'].str.split().str.len()==1].index 
two_word = df_eng[df_eng['tweet'].str.split().str.len()==2].index 
df_final =  df_eng.drop(one_word, axis = 0)
df_final =  df_eng.drop(two_word, axis = 0)

In [32]:
# df_final = df_final[(df_final['tweet'] != 'thanks yes have done') & 
#          (df_final['tweet'] != 'thanks ive done it now') & 
#          (df_final['tweet'] != 'ive followed you now') &
#          (df_final['tweet'] != 'fait') &
#          (df_final['tweet'] != 'great thank you') &
#          (df_final['tweet'] != 'thisisviolence') &
#          (df_final['tweet'] != 'done') &
#          (df_final['tweet'] != 'aaaah i want my right nowwwww') &
#          (df_final['tweet'] != 'let ya know fridentityay    ') &
#          (df_final['tweet'] != 'weeks for me') &
#          (df_final['tweet'] != 'sure') &
#          (df_final['tweet'] != 'fortunately') &
#          (df_final['tweet'] != 'what is a follow') &
#          (df_final['tweet'] != 'my phone huawei mate pro') &
#          (df_final['tweet'] != 'yup and just found it') &
#          (df_final['tweet'] != 'check this tweet') &
#          (df_final['tweet'] != 'n have left uk i think') &
#          (df_final['tweet'] != 'well it is the case') &
#          (df_final['tweet'] != 'excuse you this is wild what can possibly be mb') &
#          (df_final['tweet'] != 'how dare you') &
#          (df_final['tweet'] != 'im having this issue also but with  ') &
#          (df_final['tweet'] != 'welcome dans la famille ') &
#          (df_final['tweet'] != 'i have no private message   ') &
#          (df_final['tweet'] != 'have you tried')]

In [33]:
df_final.shape

(107037, 23)

#### Correct misspelled words noticed in data review process

In [24]:
remove_misspelled = (lambda x: 
                 x.replace('seriusly', 'seriously')
                 .replace('incomewhich', 'income which')
                 .replace('revolutteam', 'revolut team')
                 .replace('revolutuserswantscrypto', 'revolut users want crypto')
                 .replace('chatwhere', 'chat where')
                 .replace('loungesthat', 'lounges that')
                 .replace('moreexcuses', 'more excuses')
                 .replace('relovut', 'revolut')
                 .replace('nopromisedphonecall', 'no promised phone call')
                 .replace('complient', 'compliant')
                 .replace('hapenned', 'happened')
                 .replace('problembut', 'problem but')
                 .replace('sitesstill', 'site still')
                 .replace('moneylaundring', 'money laundering')
                 .replace('changeslogin', 'changes login')
                 .replace('transactioned', 'transaction')
                 .replace('reasonsprovided', 'reasons provided')
                 .replace('blockingaccount', 'blocking account')
                 .replace('whrite', 'right')
                 .replace('againshame', 'again shame')
                 .replace('moneygood', 'money good')
                 .replace('premim', 'premium')
                 .replace('pinwont', 'pinwont')
                 .replace('resetwont', 'reset wont')
                 .replace('resetfrustrating', 'reset frustrating')
                 .replace('moneywithout', 'money without')
                 .replace('conserns', 'concerns')
                 .replace('exchanses', 'exchanges')
                 .replace('plattform', 'platform')
                 .replace('informaton', 'information')
                 .replace('generatingsaving', 'generating saving')
                 .replace('breakthesilence', 'break the silence')
                 .replace('monthsampnosupportampnorefund', 'months and no support and no refund')
                 .replace('customerservicemissing', 'customer service missing')
                 .replace('verifiacationcould', 'verification could')
                 .replace('regulatinons', 'regulations')
                 .replace('fraudulence', 'fraudulent')
                 .replace('imsurqnce', 'insurance')
                 .replace('accountmoney', 'account money')
                 .replace('resolutioncustomer', 'resolution customer')
                 .replace('consumercomplaint', 'consumer complaint')
                 .replace('teamim', 'team im')
                 .replace('thant', 'than')
                 .replace('reccomand', 'recommend')
                 .replace('billsrent', 'bills rent')
                 .replace('badcustomerservice', 'bad customer service')
                 .replace('idverification', 'id verification')
                 .replace('mkwny', 'money')
                 .replace('amp', 'and')
                 .replace('wault', 'vault')
                 .replace('mths', 'months')
                 .replace('ppls', 'peoples')
                 .replace('reaponse', 'response')
                 .replace('thx', 'thanks')
                 .replace('supportannounce', 'support announce')
                 .replace('hrs', 'hours')
                 .replace('accounti', 'account i')
                 .replace('infos', 'information')
                 .replace('info', 'information')
                 .replace('weeek', 'week')
                 .replace('ltsighgt', '')
                 .replace('useing', 'using')
                 .replace('revoiut', 'revolut')
                 .replace('tchat', 'chat')
                 .replace('coldnt', 'couldnt')
                 .replace('buyinvest', 'buy invest')
                 .replace('id', 'identity')
                 .replace('criptos', 'cryptos')
                 .replace('robindhood', 'robinhood')
                 .replace('resetcodes', 'reset codes')
                 .replace('incorrecthelp', 'incorrect help')
                 .replace('againwtf', 'again wtf')
                 .replace('phishingsms', 'phishing sms')
                 .replace('wrongplease', 'wrong please')
                 .replace('compromisedhelp', 'compromised help')
                 .replace('appfrozen', 'app frozen')
                 .replace('rainbowrevolutcard', 'rainbow revolut card')
                 .replace('refferral', 'referral')
                 .replace('wealthmanagement', 'wealth management')
                 .replace('varification', 'verification')
                 .replace('coronai', 'corona')
                 .replace('accouts', 'accounts')
                 .replace('pandemicshame', 'pandemic shame')
                 .replace('autocloses', 'auto closes')
                 .replace('impower', 'empower')
                 .replace('impowermovement', 'empower movement')
                 .replace('didentityentityentityentity', '')
                 .replace('identityentityentityentity', ''))

df_final['tweet'] = df_final.tweet.map(remove_misspelled)

In [25]:
df_final.to_csv('train.csv')

In [26]:
tweets = [tweet for tweet in df_final['tweet']]

### Lemmatize, Tokenize and count words

Note that this is separate to tokenization above - what is correct in terms of process?

In [93]:
from nltk.util import ngrams

from collections import Counter

from nltk.corpus import stopwords

from textblob import TextBlob

In [94]:
stop = stopwords.words('english')
stop.extend(['ive', 'im', 'itd', 'youre', 'hi', 'hello', 'hey', 'eg', 'l', 'h','w','v','u'])

In [118]:
lemma = nltk.stem.WordNetLemmatizer()
counter = Counter()

for tweet in x_tr['tweet']:
    tweet = lemma.lemmatize(tweet)
    tweet = TextBlob(tweet).words  # tokenize words
    tweet = [w for w in tweet if w not in stop]   
    counter += Counter(tweet)

In [112]:
len(counter)

29419

In [114]:
for phrase, count in counter.most_common(100):
    print('%20s %i' % ("".join(phrase), count))

             account 24826
             revolut 14460
                 app 13548
               money 12510
              please 10976
                help 9927
                 get 8651
                cant 8088
                chat 7543
                card 6633
                need 6629
               still 6547
              thanks 5322
                dont 5281
                 one 5037
             support 4954
              access 4930
             blocked 4629
                bank 4546
                days 4320
             waiting 4153
                back 4002
              locked 3904
             service 3903
                 use 3828
            customer 3779
                like 3770
               since 3731
                 new 3688
             contact 3680
               agent 3635
                time 3257
               would 3208
               phone 3205
             someone 3100
                  dm 3085
                 got 2940
               issue 2897
       